In [3]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [4]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
# options.add_argument("disable-gpu")

## camping site 정보 수집 (부대시설 정보 추가해야함)

In [3]:
chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?pageUnit=2500&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1'

def get_siteinfo(url):
    driver.get(url) 

    time.sleep(10)

    source = driver.page_source 
    data = bs(source, 'html.parser')
    cont = data.findAll("div",{"class":"camp_cont"})

    cols = ['name','url','addr']
    df = pd.DataFrame(columns=cols)
    prefix = 'https://www.gocamping.or.kr'
    for i in range(len(cont)):
        name = cont[i].h2.a.get_text().split("]")[1]
        url_t = cont[i].h2.a['href'].split("&")[0]
        url = ''.join([prefix,url_t])
        addr = cont[i].ul(class_="addr")[0].get_text()

        info = [name, url, addr]
        dict1 = dict(zip(cols, info))
        df = df.append(dict1, ignore_index=True)

    df['name'] = df.name.str.strip()
#     df['hashtag'] = df.name.str.replace(' ','')
#     df['hashtag'] = df.hashtag.str.replace('(주)','')
    return df

## Get xy coordinate directly from embeded navermap

In [4]:
def get_sitexy(bsobj):
    cont = bsobj.findAll("div", {"class":"map_list"})[0]
    ul = cont.ul.select("li[onclick]")
    cols = ['name','addr', 'call_num','coord_x','coord_y']
    df = pd.DataFrame(columns=cols)

    for i in range(len(ul)):
        camp = ul[i]
        xy = camp['onclick']
        x = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[1]
        y = re.findall('[0-9]*[0-9]\.[0/-9]*[0-9]', xy)[0]
        name = camp.h2.text.split(']')[1].strip()
        addr = camp.find("li", {"class":"addr"}).get_text()
        call_num = camp.find("li", {"class":"call_num"}).get_text() if camp.find("li", {"class":"call_num"}) else ""
        val_list = [name, addr, call_num, x, y]
        result = dict(zip(cols, val_list))
        df = df.append(result, ignore_index=True)
    print(df.head(2))
    return df

In [5]:
cols = ['name','addr', 'call_num','coord_x','coord_y']
df_fn = pd.DataFrame(columns=cols)
limit_ = 1000

chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)

for i in range(1,4):
#     print(i)
    url = 'https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit={limit}&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex={page}'.format(limit=limit_, page=i)
    print(url)
    driver.get(url) 
    time.sleep(3)
    source = driver.page_source 
    data = bs(source, 'html.parser')
    
    df_rslt = get_sitexy(data)
#     print(df_rslt.head())
    df_fn = df_fn.append(df_rslt)
    

https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=1
            name                 addr       call_num      coord_x     coord_y
0  강원도양양캠핑장 포레스트   강원 양양군 현북면 검소길 117  010-2000-4949  128.6916045  38.0077282
1        원주키즈캠핑장  강원도 원주시 신림면 솔치로 88                  128.2027943  37.2640568
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=2
           name                    addr       call_num      coord_x  \
0  동강전망휴양림오토캠핑장  강원 정선군 신동읍 동강로 916-212  070-4225-2336  128.6079023   
1     금강산랜드 야영장     경기도 파주시 월롱면 도감로 130   031-949-6565  126.8001973   

      coord_y  
0  37.2595994  
1  37.7924796  
https://www.gocamping.or.kr/bsite/camp/info/list.do?listTy=MAP&pageUnit=1000&searchKrwd=&listOrdrTrget=last_updusr_pnttm&pageIndex=3
        name                      addr       call_num      coord_x     coord_y
0  주천강 자연휴양림  강원도 횡성군 둔내면 

In [6]:
df = df_fn.reset_index(drop=True)

In [7]:
def name_for_tag(x):
    if x=="Najost Camp": return "나조스트캠핑장"
    elif x=="iFA캠프운악": return "캠프운악"
    elif x=="Black tree": return "블랙트리캠핑장"
    elif x=="영천 구룡산 오토&펜션": return "영천구룡산오토캠핑장"
    elif x=="(주)자연에너지-드림랜드오토캠핑장": return "드림랜드오토캠핑장"
    elif x=="소회산리멍우리협곡관광농원(캠핑장)": return "멍우리협곡캠핑장"
    elif x=="춘천Believing(빌리빙)캠핑장": return "춘천빌리빙캠핑장"
    elif x=="변산반도국립공원 고사포 야영장 (임시)": return "고사포야영장"
    elif x=="A&J오토캠핑장": return "AJ오토캠핑장"
    elif x=="Camp 1950 by 민들레울": return "camp1950"
    elif x=="(주)자연인": return "자연인글램핑"
    elif x=="(주)수동자연마을 힐링별밤수목원캠핑장": return "힐링별밤수목원캠핑장"
    elif x=="stay714": return "스테이714"
    elif x=="선녀와나무꾼": return "선녀와나무꾼캠핑장"
    elif x=="블루스카이": return "블루스카이캠핑장"
    elif x=="CLUB 596": return "클럽596"
    elif x=="비토애": return "비토애글램핑"
    elif x=="(주)연합진흥 경도 글램핑파크": return "경도글램핑파크"
    elif x=="(주)태평소금 천일염 힐링캠프": return "천일염힐링캠프"
    elif x=="누룽지": return "누룽지캠핑장"
    elif x=="포시즌": return "포시즌캠핑장"
    elif x=="마이웨이 리조트 관광": return "마이웨이리조트"
    elif x=="바오바오": return "바오바오글램핑" 
    elif x=="블루마운틴": return "팔공산블루마운틴" 
    elif x=="별헤는 밤": return "별헤는밤캠핑장"
    elif x=="선녀와 나무꾼": return "선녀와나무꾼캠핑장"
    elif x=="테르메덴": return "테르메덴카라반"
    elif x=="캠프통 아일랜드": return "캠프통아일랜드글램핑"
    elif x=="포세이돈": return "포세이돈카라반"# 2266	(주)포세이돈	20904
    elif x=="노을캠핑장": return "강화노을캠핑장"# 1005	노을캠핑장	18276

    elif "&" in x: return re.sub("&","앤", x)
    elif re.search("\(.*\)", x): return re.sub("\(.*\)","", x)
    else: return x
#     else: return x.replace(" ","").strip()
        
df["hashtag_test"] = df["name"].apply(name_for_tag)
df["hashtag_test"] = df["hashtag_test"].apply(name_for_tag)
df["hashtag"] = df["hashtag_test"].apply(lambda x: x.replace(" ","").strip())
print('modified : {count}'.format(count=len(df[df["name"]!=df['hashtag_test']])))
df[df["name"]!=df['hashtag_test']].head()

modified : 90


,name,addr,call_num,coord_x,coord_y,hashtag_test,hashtag
3,성동 서울숲 여름캠핑장(임시캠핑장),서울 성동구 뚝섬로 273 (성수동1가),02-2286-5663,127.0417991,37.5430701,성동 서울숲 여름캠핑장,성동서울숲여름캠핑장
119,얼음골한옥펜션&캠핑장,경남 밀양시 산내면 하양지안길 11-19,,128.9528490,35.5984747,얼음골한옥펜션앤캠핑장,얼음골한옥펜션앤캠핑장
129,진부(오대산)관광농원,강원 평창군 진부면 아차골길 34-15,010-6343-6516,128.5672147,37.5555278,진부관광농원,진부관광농원
144,소백산 신선펜션 & 캠핑,충북 단양군 가곡면 새밭로 474,010-4963-7169,128.4228607,36.9958373,소백산 신선펜션 앤 캠핑,소백산신선펜션앤캠핑
185,(주)아웃오브파크,강원도 춘천시 남면 가옹개길 52-9,1522-1861-,127.5112565,37.7278127,아웃오브파크,아웃오브파크


In [43]:
to_drop = ['산책','선인장','부에노스아이레스','가온','캐빈','숲속의 작은집','생각속의 집', '풍경']
df = df[~df.name.isin(to_drop)]
df.reset_index(drop=True, inplace=True)

In [44]:
len(df['name'].unique())

2398

In [45]:
df.to_csv("../file/campingsite.csv")

In [50]:
df = pd.read_csv("../file/campingsite.csv", index_col=[0])

## 페이지 내에서 검색어 입력 -> 클릭 -> 검색어 입력 -> 클릭... 방식

In [51]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("disable-gpu")

In [ ]:
# You are getting a list of web elements with
cols = ['num_index','hashtag', 'search_order', 'search_rslt', 'url','text']
df_out = pd.DataFrame(columns=cols)

chrome_path = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chrome_path, options=options)
driver.get('https://www.instagram.com/explore/tags/'+df['hashtag'][0]+'/?hl=en') 

time.sleep(2)

t = driver.find_elements_by_css_selector("input[placeholder='Search']")[0]
i = 0
# for i in range(2):
for i in range(len(df)):
    print(i, end=" ")
    t.send_keys(df['hashtag'][i])
    time.sleep(1.5)
    t2 = driver.find_elements_by_class_name("yCE8d")
    for j in range(len(t2)):
        ref = t2[j].get_attribute('href')
        title = t2[j].text
        txt = t2[j].find_elements_by_class_name("Fy4o8")[0].text
        dict1 = dict(zip(cols, [i, df['hashtag'][i], j, title, ref, txt]))
        df_out = df_out.append(dict1, ignore_index=True)
    t.clear()
    if i>0 and i%100 == 0:
        print(df_out.tail(2))
driver.quit()
df_out.head(20)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100     num_index      hashtag search_order           search_rslt  \
395        99       자연속캠핑장            4   #자연속캠핑장넘좋다👍\n1 post   
396       100  청도감꽃국민여가캠핑장            0  #청도감꽃국민여가캠핑장\n1 post   

                                                   url    text  
395  https://www.instagram.com/explore/tags/%EC%9E%...  1 post  
396  https://www.instagram.com/explore/tags/%EC%B2%...  1 post  
101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179

In [ ]:
today_ymd = datetime.datetime.today() .strftime('%Y%m%d')
df_out.to_csv('../file/camping_' + today_ymd + '.csv', index=False)

In [25]:
driver.quit()

# Instagram Hashtag 수집

In [27]:
def get_postnum(df, tag_nm, num=0):
    today = datetime.datetime.today() 
    today_ymd = today.strftime('%Y%m%d')
    
    cols = list(df.columns) + ['num_posts','date_time']
    df_out = pd.DataFrame(columns=cols)
    
    chrome_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_path, options=options)

    for i in range(num,len(df)):
        driver.get('https://www.instagram.com/explore/tags/'+df[tag_nm][i]+'/?hl=en') 
        time.sleep(2.5) 
        source = driver.page_source 
        data = bs(source, 'html.parser') 
        tag = data.find("span",{"class": "g47SY "}) 
        try: 
            val = int(tag.text.replace(',' , '')) 
        except AttributeError: 
            val = 0

        info = list(df.iloc[i,:]) + [val, today]
        dict1 = dict(zip(cols,info))
        df_out = df_out.append(dict1, ignore_index=True)
        print(i,end=" ")

        if i>0 and i%100 == 0:
            print(df_out.tail(1))
            driver.quit()
            time.sleep(10)
            driver = webdriver.Chrome(chrome_path, options=options)
    driver.quit()
    return df_out


In [28]:
df_postnum = get_postnum(df, 'hashtag')

KeyboardInterrupt: 

In [25]:
driver.quit()
time.sleep(10)
driver.get('https://www.instagram.com/explore/tags/'+df['hashtag'][0]+'/?hl=en') 

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=53921): Max retries exceeded with url: /session/e79022d7610e4e1615451ce7288d9a89/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f93d9e434a8>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [822]:
today_ymd = datetime.datetime.today() .strftime('%Y%m%d')
df_postnum.to_csv('../file/camping_' + today_ymd + '.csv', index=False)

In [825]:
df_postnum.sort_values(by='num_posts', ascending=False)[['name','hashtag','num_posts']].head(20)

,name,hashtag,num_posts
216,중랑캠핑숲,중랑캠핑숲,7645
1139,느랭이골,느랭이골,3229
2258,YELLOW HILL,YELLOWHILL,894
2306,마인드 글램퍼스,마인드글램퍼스,750
1814,파인트리 캠핑장,파인트리캠핑장,397
1151,돌산캠핑장,돌산캠핑장,359
1901,흥정계곡 캠핑 700,흥정계곡캠핑700,343
2134,브라보캠핑장,브라보캠핑장,330
2256,가족쉼터,가족쉼터,324
2245,고대산캠핑리조트,고대산캠핑리조트,180


In [826]:
df_postnum[df_postnum['hashtag']=='캠프운악']

,name,addr,call_num,coord_x,coord_y,hashtag,hashtag_test,num_posts,date_time
351,iFA캠프운악,경기도 포천시 화현면 화현리 1090-2,010-2923-8141,127.3058131,37.8946656,캠프운악,캠프운악,0,2020-06-19 00:19:01.778953


## get post url

## geocoding example

In [ ]:
cols = list(df.columns) + ['level1','level2','x','y']
df_fn = pd.DataFrame(columns=cols)
apikey = '6829FE20-AE45-312E-9D0A-418C7E77BF47'

for i in range(len(df)):
    dict0 = dict(df.iloc[i])
#     print(dict0)

    address = df['addr'][i]
    URL = 'http://api.vworld.kr/req/address?service=address&request=getcoord&version=2.0&crs=epsg:4326&address={addr}&refine=true&simple=false&format=json&type=road&key={key}'.format(key=apikey, addr=address)
    r = requests.get(URL)
    data = r.json()
    try: 
        dict1 = data['response']['refined']['structure']
        dict1 = {k: dict1[k] for k in dict1.keys() & {'level1','level2'}}
        dict2 = data['response']['result']['point']
        dict0.update(dict1)
        dict0.update(dict2)
    except KeyError:
        dict1 = {'level1':None, 'level2':None, 'x':None, 'y':None}
        dict0.update(dict1)
    
    df_tmp = pd.DataFrame(json_normalize(dict0), columns=dict0.keys())
    df_fn = df_fn.append(df_tmp)
    print(df_fn.tail(5))